# Resonance Elastic Scattering of $\alpha$-particles with ${}^{10}\text{C}$

In [2]:
from pathlib import Path
import json

The GET acquisition system serialises data using the MultiFrame Metaformat (MFM)
> a binary format for data acquisition and serialization that are self-contained, layered, adapted to network transfers[,] and evolving. {cite:ps}`anvar_multiframe_nodate`

This file format is more suitable for disk storage than in-memory representation of data, as it is a heirarchical format in which each _frame_ must be read, regardless of whether it is needed. The use of separate header and data sections means that the cost of skipping a frame is limited to parsing the header, but MFM is still a struct-based format. In many cases, columnar processing is both more performant and more expressive than an equivalent event-loop based approach.{cite:ps}`smith_case_2019` 

The analysis process was performed in a series of discrete stages (see {numref}`analysis-flowchart`).

<!-- :::{figure-md} analysis-flowchart -->

:::{mermaid}
flowchart LR

signal --> calibrate[Silicon Calibration*]
baseline_removal --> gain_matching[Gain Matching*]
baseline_removal --> response[Response Estimation*]
gain_matching --> signal
calibrate --> kinematic_fitting
response --> signal

get[/GET MFM/] --> mfm2root[ROOT Conversion] --> partitioning[Partitioning] --> noise_removal[Noise Removal] --> baseline_removal[Baseline Removal] --> signal[Signal Fitting] --> event[Cluster Reconstruction] --> fit[Track Fitting] --> kinematic_fitting[Kinematic Fitting] --> kinematics[/Kinematics/]

gas_parameters[/Gas Parameters/] --> gas_simulation[Garfield++ Simulation] --> event

:::

<!-- The asterisk \* indicates processes which were aggregated over the entire dataset ahead of reconstruction -->
<!-- ::: -->

```{warning}
TODO: 
- [ ] make this wrap nicely
- [ ] support hyperlinks to sections in this figure
```

## Partitioning
At the earliest stages of the reconstruction pipeline, performance is primarily bounded by available memory, and therefore it is sensible to partition the data by event multiplicity. This process constitutes the first two stages of this analysis, seen in {numref}`analysis-flowchart`. In the ROOT dataset, the `mmMul` branch contains the per-event multiplicity. To regularise the dataset according to this multiplicity, it was partitioned into approximate chunks of a given aggregate multiplicity. Given that the memory of an MFM event is primarily determined by the number of waveforms that it contains, this effectively balances the memory footprint of each partition.

In subsequent stages, the memory bound gives way to compute costs, and so the partition size is refined to balance naive parallelisation against parallisation overhead. Where data is written to disk, the Hierarchical Data Format version 5 (HDF5) format is used as a consequence of its good language support, high compression ratios (with GZip), and columnar representation.{cite:ps}`the_hdf5_group_hierarchical_1997` The Parquet format is another strong format for columnar representation but, due to some library constraints, HDF5 was preferred.

## Noise and Baseline Removal

As discussed in {ref}`content:get-architecture`, the AGET chip provides four additional channels for measuring the electronic noise. These channels can be used to eliminate some of the electronic noise inherent within the system. A number of noise removal strategies are discussed in {cite:ps}`giovinazzo_get_2016-1`, and establishes that an average over the set of FPN channels per-chip is optimal. It was this strategy that was employed to for noise removal (see {numref}`fpn-removal-micromegas`).


:::{figure} todo
:name: fpn-removal-micromegas
:width: 400px
:align: center

```{warning}
Todo: add example of FPN channel & FPN removal
```
:::

In addition to a low frequency component of the electronic noise, a separate baseline offset was observed in the MicroMeGAS and silicon detector channels. This component was removed using two separate methods:

### MicroMeGaS Baseline Estimation

Given that time in the MicroMeGaS is primarly determined by the drift time of the liberated electrons in the gas volume, the baseline observed in the MicroMeGaS detector was not seen to occur at specific times. Therefore, in order to identify the region of baseline that would be linearly subtracted from the recorded signal, a local variance was computed for the channel samples, with a window of 96 cells (see {numref}`baseline-estimate-micromegas`).

:::{figure} todo
:name: baseline-estimate-micromegas
:width: 400px
:align: center

```{warning}
Todo: add example of baseline estimation
```
:::


The value of the baseline was given by the locus of the minimum computed variance.

### Silicon Baseline Estimation

Unlike the MicroMeGaS channels, the silicon quadrant detectors have a well defined boundary between the baseline and the interaction signals. This boundary occurs at a similar time for each detector, as it is the silicon detectors upon which the GET system was configured to trigger. To determine the baseline value, the mean of the sample values for the preceeding ten cells was used (see {numref}`baseline-estimate-silicon`).

:::{figure} todo
:name: baseline-estimate-silicon
:width: 400px
:align: center

```{warning}
Todo: add example of baseline estimation
```
:::

## Response Estimation

Channels which employ the AGET shaper require deconvolution in order to recover the underlying signal. This process requires a good knowledge of the underlying response function of the system. An approximate analytical solution is given in {cite:ps}`giovinazzo_get_2016-1`, but the authors note that it is insufficiently accurate to be used with deconvolutional methods. It is noted that the electronics can be configured such that the response function can be directly measured, but this required secondary access to the detector which resides at the Cyclotron Institute, Texas A&M University.{cite:ps}`giovinazzo_get_2016-1` 

### Iterative Estimation
In order to determine a better approximation for the response function, an iterative deconvolve-fit process can be applied to a set of MicroMeGaS waveforms. These waveforms should be taken from a random spanning sample of the recorded MicroMeGaS waveforms. 

:::{figure} todo
:name: mm-random-sample
:::

:::{warning}
Figure showing random MM sample
:::

Given such a sample, an initial estimate of the response function $F^{(1)}$ can be used to fit each waveform with the convolution of a Gaussian function 

:::{math}
:label: iterative-estimate
Y^{(1)} = F^{(1)} * \mathcal{N}^{(1)}\,.
:::

The resulting fit $\mathcal{N}^{(1)}$ can then be _deconvolved_ from the original sample $Y$ to produce of $F^{(2)}$. This process can be repeated until the solution becomes stable, and the solutions for a set of random waveforms may be averaged to produce a singular response function $F$ (see {numref}`mm-response-vs-get`).

:::{figure} todo
:name: mm-response-vs-get
:::

:::{warning}
Figure showing MM response vs GET
:::


Given that the convolution of any two Gaussians is also Gaussian, the solution from {eq}`iterative-estimate` is not guaranteed to be unique, i.e. the resulting response function may include some Gaussian component. This can be seen when the estimated response function is compared with any one silicon sample: the silicon sample is on average slightly narrower than that of the estimated response function.


:::{figure} todo
:name: mm-vs-si-sample
:::

:::{warning}
Figure showing comparing MM and si widths
:::

### Sample Averaging

It follows that the silicon waveforms may provide a more robust mechanism for estimating the response function. If one assumes that that the shape of the current signal produced by interactions with the silicon detector is predominantly determined by the shaper, then the measured signal is to first approximation simply a convolution of the intrinsic response function and some normal component $F' * \mathcal{N}$. 

To build a response function estimate, a random sample of silicon waveforms spanning a range of amplitudes is taken. The binwise mean of this set of waveforms is taken to produce an average waveform. This waveform is an approximate response function, which can then be time-shifted such that the peak coincides with $t=0$.


:::{figure} todo
:name: si-average-sample
:::

:::{warning}
Figure showing random si mean sample
:::

## Gain Matching

In {ref}`content:micromegas`, it was noted that the MicroMeGaS anode is subdivided into a set of distinct zones that can be held at different potentials in order to spatially vary the gain. In this experiment, both the strip-chain (side) regions and final block of pads in the central region were held at a high gain in order to better resolve light particle tracks with low stopping powers (see {numref}`micromegas-anode-gain-region`). 

:::{figure} todo
:name: micromegas-anode-gain-region
:::

:::{note}
TODO: add an image of micromegas-anode-gain-region
:::

Following position reconstruction, these gains must be accounted for if the collected charge is to be used for particle identification. Given that the strips, chains, and final central pads were held at the same potential, the relative gain of these regions can be determined solely by looking at the relative gain of the final central pad region. This was experimentally measured by observing the change in charge collected by the rows of pads either side of the high-low gain boundary.


:::{figure} image/placeholder/gain-high-dark.png
---
alt: Ratio of low gain and high gain mean charge at the gain boundary.
width: 400px
align: center
---
Ratio of the mean collected charge between the low gain and high gain region boundary in the Micromegas pads.
:::

In [6]:
# Load the relative gain
relative_gain = json.loads((Path("data") / "relative-gain.json").read_text())
relative_gain_3f = f"{relative_gain:.3f}"

After fitting this distribution with a binned likelihood estimator, the relative gain between the high and low gain regions was found to be {eval}`relative_gain_3f`.

## Signal Fitting

In the above sections, it was established that the sampled GET waveforms are effectively convolutions of an original source signal and the intrinsic GET shaper response function. Having determined an approximation for this response function, the source signals can then be recovered by deconvolutional methods. 

For each preprocessed sample, a convolutional Gaussian fit is performed that solves $y = F * \mathcal{N}(\boldsymbol{\phi})$ for $\boldsymbol{\phi}$. This is preferred over computing the Gaussian fit of the deconvolved signal; although repeated convolution is computationally expensive, it is less vulnerable to artifacts produced by the deconvolution technique. For samples which do not exceed the dynamic range of the acquisition system, the peak multiplicity is estimated using the GOLD deconvolution algorithm with boosting, which collapses broad peaks into smooth, sharp peaks. These peaks are then identified using a simple turning-point localiser, which is used to seed the intial fit parameters (see {numref}`turning-point-localiser`). 

:::{figure} todo
:name: turning-point-localiser
:::

:::{warning}
Figure show turning point localiser
:::

In some cases, the measured sample exceeds the maximum value that can be measured. This saturation introduces a discontinuity in the derivative of the sample, which renders it unamenable to deconvolution (see {numref}`signal-saturation-discontinuity`).
:::{figure} todo
:name: signal-saturation-discontinuity
:::

:::{warning}
Figure show signal-saturation-discontinuity
:::

In such instances, one cannot easily determine the peak multiplicity of the sample. It is therefore assumed that the measured sample contains only one peak, which is fit as described above using an additional mask that forces the fit algorithm to reject the saturated regions.


:::{warning}
TODO: 
- [ ] discuss current vs voltage here of preamplifier
- [ ] check whether drift of charges is signficant to signal formation in presence of micromesh?
- [ ] consider meaning of non linear least squares
:::

## Silicon Calibration

- $n$ alpha source with energies ...
- found resolution
- use of mass to resolve broad peaks for bad dets

## Garfield++ Simulation

## Cluster Reconstruction

## Track Fitting

## Kinematic Fitting

## Beam